In [1]:
# automatically update imports
%load_ext autoreload
%autoreload 2

# For graphs
%matplotlib inline

# snippet for relative imports
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
# load necessary data
import numpy as np

import osm_loader

step = 0.0025

city_refs = [
    'begur',
    'grenoble',
    # 'san_francisco',
    'fort_collins',
    # 'sant_feilu',
    # 'sierra_vista'
]

bboxes = { city_ref : bbox for city_ref, bbox in zip(city_refs, [
    [41.9301, 3.1817, 41.9751, 3.2389],
    [45.1334, 5.6127, 45.2345, 5.8238],
    # [37.68, -122.5346, 37.8163, -122.3431],
    [40.4616, -105.1783, 40.6461, -104.9716],
    # [41.3700, 2.0203, 41.3973, 2.0774],
    # [31.5109, -110.3800, 31.5816, -110.2052]
]) }

poiss = { city_ref : osm_loader.pois_from_bbox(bboxes[city_ref]) for city_ref in city_refs }

In [4]:
# missing data sensitivity
import kde

missing_percentages = np.arange(0, .3, .05)

missing_poiss = {}
missing_kdes = {}
for city_ref in city_refs:
    city_pois = {}
    city_kdes = {}
    bbox = bboxes[city_ref]
    pois = poiss[city_ref]
    for missing_perc in missing_percentages:
        pois_missing = pois.drop(np.random.choice(pois.index, int(missing_perc*len(pois))))
        city_pois[missing_perc] = pois_missing
        city_kdes[missing_perc] = kde.get_grid_category_kde(pois_missing, bbox, step)
    missing_poiss[city_ref] = city_pois
    missing_kdes[city_ref] = city_kdes 

In [5]:
# missing data sensitivity
import pandas as pd

import kde
import spatial_measures
from utils import grid_from_bbox

missing_percentages = np.arange(0, .3, .05)

measure_methods = { 'geary' : spatial_measures.adjusted_geary, 'moran' : spatial_measures.moran_index }
measure_pre_methods = { 'count' : spatial_measures.grid_cell_pois_count, 'kde' : spatial_measures.grid_cell_kde_average }
measure_series = {}

for city_ref in city_refs:
    print(city_ref)
    city_measures = {}
    bbox = bboxes[city_ref]
    city_pois = missing_poiss[city_ref]
    city_kdes = missing_kdes[city_ref]
    for measure_key, measure_method in measure_methods.items():
        measure_ser = {}
        for pre_key, pre_method in measure_pre_methods.items():
            pre_ser = []
            for missing_perc in missing_percentages:
                xx, yy = grid_from_bbox(bbox, step)
                if pre_key == 'count':
                    ff = pre_method(city_pois[missing_perc], xx, yy)
                else:
                    ff = pre_method(city_kdes[missing_perc].values)
                pre_ser.append(measure_method(ff, xx, yy))
            measure_ser[pre_key] = pd.Series(pre_ser, index=missing_percentages)
        city_measures[measure_key] = measure_ser
    measure_series[city_ref] = city_measures

fort_collins


grenoble


begur


In [8]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

n_cities = len(city_refs)
n_measures = len(measure_methods)

pre_colors = cm.jet_r(np.linspace(0,1,len(measure_pre_methods)))

%matplotlib qt
fig, axes = plt.subplots(n_measures, n_cities, figsize=(4 * n_cities, 4 * n_measures))
[ax.set_title(city_ref) for city_ref, ax in zip(city_refs, axes[0])]
[axs[0].set_ylabel(measure_key, rotation=90) for measure_key, axs in zip(measure_methods.keys(), axes)]
for measure_key, axs in zip(measure_methods.keys(), axes):
    print(measure_key)
    for city_ref, ax in zip(city_refs, axs):
        print(city_ref)
        # city_sample_n = [activity_step_kdes[city_ref][step].size for step in steps]
        city_measure = measure_series[city_ref][measure_key]
        for i, pre_key in enumerate(measure_pre_methods.keys()):
            print(pre_key, city_measure[pre_key].var())
            ax.plot(missing_percentages, city_measure[pre_key], color=pre_colors[i], label=pre_key)
        ax.set_xlabel('n_samples')
            
plt.show()

moran
begur
('count', 3.3438973504491005e-06)
('kde', 1.0171886573403611e-06)
grenoble
('count', 9.7542775889720754e-08)
('kde', 3.4794346336188778e-07)
fort_collins
('count', 6.7965816057710185e-07)
('kde', 1.2604545758326601e-05)
geary
begur
('count', 3.6111155002979878e-06)
('kde', 0.0014221947509593256)
grenoble
('count', 6.3128246258644912e-08)
('kde', 4.5464848216160447e-07)
fort_collins
('count', 2.8705560142021131e-06)
('kde', 9.7406653528033568e-06)
